In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chisquare
from scipy.stats import t, ttest_1samp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

url = 'https://raw.githubusercontent.com/dsguseong/Mystudy/main/vgames2.csv'
df = pd.read_csv(url)
df.head()
df.drop('Unnamed: 0', axis = 1, inplace=True) # 중복된 인덱스 열 제거 후 적용

#df = df.infer_objects()
df.info() #결측지 확인하기


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.1+ MB


In [ ]:
df.isnull() #결측지 확인

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
16593,False,False,False,False,False,False,False,False,False
16594,False,False,False,False,False,False,False,False,False
16595,False,False,False,False,False,False,False,False,False
16596,False,False,False,False,False,False,False,False,False


In [ ]:
df.isnull().sum()  #각 칼럼별 결측지 수 총합 확인

Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [ ]:
df = df.dropna(how='any') #결측지 수가 데이타 수에 비하여 미비하기에 제거하여도 무방하다고 판단, 결측지가 하나라도 있는 행 모두 제거, 총 제거 수 : 379개
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [ ]:
df.info() #결측지 제거 후 데이터 정보

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16241 non-null  object 
 1   Platform     16241 non-null  object 
 2   Year         16241 non-null  float64
 3   Genre        16241 non-null  object 
 4   Publisher    16241 non-null  object 
 5   NA_Sales     16241 non-null  object 
 6   EU_Sales     16241 non-null  object 
 7   JP_Sales     16241 non-null  object 
 8   Other_Sales  16241 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.2+ MB


In [ ]:
df = df[df.Year > 1900] #1900년도 이상의 데이터만 불러오기 
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [ ]:
df_a = df.iloc[:,[0,1,2,3,4]]
df_b = df.iloc[:,[5,6,7,8]]
NA = pd.to_numeric(df_b['NA_Sales'], errors='coerce')
a = NA.dropna(how='any')
EU = pd.to_numeric(df_b['EU_Sales'], errors='coerce')
b = EU.dropna(how='any')
JP = pd.to_numeric(df_b['JP_Sales'], errors='coerce')
c = JP.dropna(how='any')
Other = pd.to_numeric(df_b['Other_Sales'], errors='coerce')
d = Other.dropna(how='any')
df_a['NA_Sales'] = a
df_a['EU_Sales'] = b
df_a['JP_Sales'] = c
df_a['Other_Sales'] = d
df0 = df_a.dropna(how='any')    # 모든 sales 열 안에 있는 K or M 값이 포함되어 있는 행의 데이터 모두 제거  
df0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0.00,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0.00,0.00,0.00
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [ ]:
#aa = df['NA_Sales'].str.contains("K")  나중에 포함된것만 분류해서 제거 하는거 방법 알아보자 지금은 시간없으니 일단 간단한걸로 넘어가장^^
#df[aa]

In [ ]:
#df['NA_Sales'].replace('M','')
#df['NA_Sales'].astype('float')
#NA = pd.to_numeric(df['NA_Sales', 'EU_Sales'], errors='coerce')

#NA = df3.dropna(how='any')
#df3.head(40)
#df3
#df['NA_Sales'].replace('K','')
#df['NA_Sales'].head(20)
#df['NA_Sales'] = pd.to_numeric(df['NA_Sales'])

In [ ]:
#df['Platform'].cat.categories

In [ ]:

df1 = df0.set_index("Name") #Name 칼럼을 인덱스로 세팅
df1

,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Name,,,,,,,,
Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00
The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01
Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00
Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00
Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04
...,...,...,...,...,...,...,...,...
Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0.00,0.01
Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0.00,0.00,0.00
NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13


In [ ]:
df1.transpose()

Name,Candace Kane's Candy Factory,The Munchables,Otome wa Oanesama Boku ni Koi Shiteru Portable,Deal or No Deal: Special Edition,Ben 10 Ultimate Alien: Cosmic Destruction,Power Pro Kun Pocket 8,JASF: Jane's Advanced Strike Fighters,Samurai Warriors: State of War,Imabikisou,The Penguins of Madagascar: Dr. Blowhole Returns - Again!,Harvest Moon: More Friends of Mineral Town,Mount & Blade,The Professor's Brain Trainer: Logic,Zoids Dash,Mini Ninjas,Rise of Nations,Hajime no Ippo: The Fighting! (2014),Grand Kingdom,Tropico Reloaded,Ty the Tasmanian Tiger,Ghost in the Shell,Cabela's Dangerous Hunts 2,Don Bradman Cricket 14,Injustice: Gods Among Us,Dr. Muto,Starwinder: The Ultimate Space Race,World of Tanks,Famicom Mini: Makaimura,McGrath Vs. Pastrana Freestyle Motocross,Kids Learn Music A+ Edition,Project Sylpheed: Arc of Deception,Mario & Luigi: Paper Jam & Mario Kart 7 Double Pack,Mega Man Anniversary Collection,G-Force,Beamrider,Ninja Reflex,FIFA World Cup Germany 2006,Vagrant Story,Knights of Honor,Phoenix Wright: Ace Attorney - Trials and Tribulations,...,Left 4 Dead,Crayon Shin-Chan Shokkugan! Densetsu o Yobu Omake Daiketsusen!!,ASH: Archaic Sealed Heat,Zen-Nippon GT Senshuken,LEGO Rock Raiders,7 Days to Die,Dragon's Crown,Terraria,The Voice,Sherlock Holmes: The Mystery of the Mummy,Wall-E,Way of the Samurai 3,Toshochu: Run for Money,Um Jammer Lammy,Yakuza,Kimikare: New Semester,Power Pro Kun Pocket,Amped 2,GoldenEye 007 (2010),Finding Nemo,Brave Story: New Traveler (US sales),Grand Theft Auto V,Yoru no Nai Kuni,Pinball Hall of Fame: The Gottlieb Collection,Super Ghouls 'n Ghosts,LEGO The Hobbit,Shanghai: True Valor,LEGO Indiana Jones 2: The Adventure Continues,Bionicle Heroes,Pokemon Channel,Jikkyou Powerful Pro Yakyuu 2000 Ketteiban,MindJack,Namco Soccer Prime Goal,White Album: Tsuzurareru Fuyu no Omoide,Minna no Chizu 2,Ice Age 2: The Meltdown,Rainbow Islands: Revolution,NBA 2K16,Toukiden: The Age of Demons,The King of Fighters '95
Platform,DS,Wii,PSP,DS,PS3,DS,PC,PSP,PS3,PS3,GBA,PC,DS,DS,PS3,PC,PS3,PS4,PC,PS2,PS,XB,X360,PS3,GC,PS,PC,GBA,PS,DS,X360,3DS,PS2,Wii,2600,Wii,XB,PS,PC,DS,...,X360,DS,DS,SNES,PS,XOne,PSV,PS3,PS3,DS,Wii,PS3,3DS,PS,PS3,PSP,GB,XB,DS,PS2,PSP,PC,PS4,PSP,GBA,XOne,PS,Wii,DS,GC,PS,PS3,PS,PS3,PSP,GC,PSP,PS3,PSV,PS
Year,2008,2009,2010,2010,2010,2005,2011,2005,2007,2011,2003,2008,2005,2006,2009,2003,2014,2015,2009,2002,1997,2005,2014,2013,2002,1996,2011,2004,2000,2011,2006,2015,2004,2009,1984,2008,2006,2000,2004,2007,...,2008,2010,2007,1995,1999,2016,2013,2013,2014,2009,2008,2008,2012,1998,2012,2012,1999,2003,2010,2003,2006,2015,2015,2005,2002,2014,1998,2009,2006,2003,2000,2011,1995,2010,2007,2006,2005,2015,2013,1996
Genre,Action,Action,Adventure,Misc,Platform,Sports,Simulation,Action,Adventure,Action,Simulation,Role-Playing,Puzzle,Role-Playing,Action,Strategy,Sports,Role-Playing,Strategy,Platform,Simulation,Sports,Sports,Fighting,Platform,Racing,Shooter,Platform,Racing,Misc,Shooter,Misc,Platform,Action,Racing,Action,Sports,Role-Playing,Strategy,Adventure,...,Shooter,Action,Role-Playing,Racing,Strategy,Action,Role-Playing,Action,Action,Adventure,Platform,Action,Adventure,Misc,Action,Action,Sports,Sports,Action,Action,Role-Playing,Action,Role-Playing,Misc,Platform,Action,Puzzle,Action,Shooter,Adventure,Sports,Shooter,Sports,Adventure,Misc,Platform,Action,Sports,Action,Fighting
Publisher,Destineer,Namco Bandai Games,Alchemist,Zoo Games,D3Publisher,Konami Digital Entertainment,Deep Silver,Tecmo Koei,Sega,THQ,Ubisoft,Paradox Interactive,505 Games,Tomy Corporation,Eidos Interactive,Microsoft Game Studios,Namco Bandai Games,Nippon Ichi Software,Kalypso Media,Electronic Arts,Sony Computer Entertainment,Activision,Tru Blu Entertainment,Warner Bros. Interactive Entertainment,Midway Games,Mindscape,Wargaming.net,Capcom,Acclaim Entertainment,Unknown,Microsoft Game Studios,Nintendo,Capcom,Disney Interactive Studios,Activision,Electronic Arts,Electronic Arts,Crave Entertainment,Sunflowers,Capcom,...,Electronic Arts,Namco B